In [1]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
import os

#request, pandas, pillow, tqdm


In [13]:
df =pd.read_csv("0_scrape_hm/articles.csv", dtype=str)
df.head()

,article_id,product_desc,product_url,image_url
0,1160705014,2-delat set i bomull - Ljusbeige/Vit/Blommig/V...,https://www2.hm.com/sv_se/productpage.11607050...,https://image.hm.com/assets/hm/8a/e3/8ae392b91...
1,1160705013,2-delat set i bomull - Vit/Blommig/Ljusbeige/V...,https://www2.hm.com/sv_se/productpage.11607050...,https://image.hm.com/assets/hm/b5/5b/b55ba1983...
2,1224620004,Mönstrad romper i muslin - Vit/Blommig/Vit/Blo...,https://www2.hm.com/sv_se/productpage.12246200...,https://image.hm.com/assets/hm/07/a8/07a8bb1b0...
3,1281158002,2-delat muslinset med omlott - Ljusbeige/Blomm...,https://www2.hm.com/sv_se/productpage.12811580...,https://image.hm.com/assets/hm/12/e3/12e36985d...
4,1218179012,Bodyklänning med volanger - Vit/Blommig/Crèmev...,https://www2.hm.com/sv_se/productpage.12181790...,https://image.hm.com/assets/hm/6a/98/6a9861989...


## Clean Data

### Remove items with no image

In [14]:
# set imwidth to make loading images faster
df.image_url = df.image_url+"?imwidth=500"


In [15]:
from PIL import Image

dir = "0_scrape_hm/images/"


valid_images = []
for index in tqdm(range(len(df))):
    subdir = f"{df['article_id'][index]}.jpg"
    path = os.path.join(dir, subdir)
    if os.path.isfile(path):
        try:
            with Image.open(path) as img:
                img.verify()  # Verify that it is a valid image
            valid_images.append(True)
        except (IOError, SyntaxError):
            valid_images.append(False)
    else:
        valid_images.append(False)

df["valid_image"] = valid_images
df_dropped=df[df["valid_image"]==True].copy()


#Reset index
print("items before dropping: ", len(df))
df_dropped.drop(columns="valid_image", inplace=True)
df_dropped= df_dropped.reset_index().drop(columns="index")
print("items after dropping: ", len(df_dropped))

  0%|          | 0/22741 [00:00<?, ?it/s]

items before dropping:  22741
items after dropping:  22729


## Save to csv

In [16]:
df_dropped.to_csv("H&M_items.csv", index=False)
df_dropped.to_csv("../api/data/H&M_items.csv", index=False) # save to the api

In [5]:
import sys
print(f"{sys.getsizeof(df_dropped)/1000000} mb")


10.419174 mb


In [6]:
df_dropped.shape

(22729, 4)